In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from mshr import *
from braininversion.DarcySolver import solve_darcy
from pyadjoint import ipopt

from  braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                                    plot_pressures_and_forces_cross_section,
                                    extract_cross_section, style_dict)

# time stepping
T = 1.2           # final time
num_steps = 12    # number of time steps
dt = T/ num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17 # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-4          # storage coefficient
mmHg2Pa = 132.32

# create mesh and mark boundaries
N = 5 # resolution
brain_radius = 0.1 
ventricle_radius = brain_radius/3
brain = Circle(Point(0,0), brain_radius)
ventricle = Circle(Point(0,0), ventricle_radius)
brain = brain - ventricle
mesh = Mesh(generate_mesh(brain, N))

ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R*0.95)",
                              R =brain_radius )
skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.95 )",
                          R = brain_radius)
boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
skull.mark(boundary_marker, 1)
ventricle.mark(boundary_marker, 2)
x_coords = np.linspace(ventricle_radius, brain_radius, 20)
slice_points = [Point(x, 0.0) for x in x_coords]

# set analytical expressions
f = 1
A = 10/(brain_radius - ventricle_radius)*mmHg2Pa
p_obs = Expression("A*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*sin(2*pi*f*t)",
                    A=A, f=f, t=0, R_vent=ventricle_radius, degree=2)

f_ana = Expression("- K*A/(sqrt(x[0]*x[0] + x[1]*x[1]))*sin(2*pi*f*t)" +
                   "+ 2*c*A*pi*f*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*cos(2*pi*f*t)",
                    K=K, A=A,c=c, f=f, t=0, R_vent=ventricle_radius, degree=2)

p_N =  Expression("A*sin(2*pi*f*t)", A=A, f=f, t=0, R_vent=ventricle_radius, degree=2)

In [ ]:
def optimize_force(K, p_obs, boundary_marker, boundary_conditions):
    control_space = FunctionSpace(mesh, "CG", 1)
    ctrls = [Function(control_space,
                  name="control") for i in range(num_steps)]

    control = [Control(c) for c in ctrls]

    solution = solve_darcy(mesh, ctrls, T, num_steps, K,
                          boundary_marker, boundary_conditions,
                          c=c, p_initial=p_obs, degree=1, theta=0.5)
    initial_solution = []
    J = 0
    dS = Measure("dS", domain=mesh)
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2.0
    for i,p in enumerate(solution):
        p_obs.t = times[i]
        #J += assemble((p_obs - p)**2*dx)
        #J += assemble(jump(grad(p), n)**2*dS )
        J+= assemble(1.0/h_avg*jump(grad(p), n)**2*dS ) # +assemble(div(grad(p))**2*dx)
        initial_solution.append(p.copy())
        
    rf = ReducedFunctional(J, control)
    problem = MoolaOptimizationProblem(rf)
    f_moola = moola.DolfinPrimalVectorSet(
        [moola.DolfinPrimalVector(c, inner_product="L2") for c in ctrls])
    
    solver = moola.BFGS(problem, f_moola, options={'jtol': 1e-12,
                                                   'gtol': 1e-9,
                                                   'Hinit': "default",
                                                   'maxiter': 50,
                                                   'mem_lim': 100})
    sol = solver.solve()
    opt_ctrls = sol['control'].data
    #problem = MinimizationProblem(rf)

    #solver = IPOPTSolver(problem, parameters={'maximum_iterations': 50})
    #opt_ctrls = solver.solve()

    opt_solution = solve_darcy(mesh, opt_ctrls, T, num_steps, K,
                              boundary_marker, boundary_conditions,
                              c=c, p_initial=p_obs, degree=1, theta=0.5)
    opt_solution = [s.copy() for s in opt_solution]
    
    return opt_ctrls, opt_solution, initial_solution

In [ ]:
# Optimize with Dirichlet bc
dirichlet_boundary_conditions = {1:{"Dirichlet":p_obs}, 2:{"Dirichlet":p_obs}}

%time res = optimize_force(K, p_obs, boundary_marker, dirichlet_boundary_conditions)
diri_opt_ctrls, diri_opt_solution, initial_solution = res

In [ ]:
# Optimize with Robin bc at skull
beta = 0.5
robin_boundary_conditions = { #1:{"Robin": (beta, beta*p_obs + p_N)},
                               1:{"Neumann": p_N},
                              2:{"Dirichlet":p_obs}}
%time res = optimize_force(K, p_obs, boundary_marker, robin_boundary_conditions)
robin_opt_ctrls, robin_opt_solution,initial_solution = res

In [ ]:
# compute pressure from analytical force
ana_solution = solve_darcy(mesh, f_ana, T, num_steps, K,
                          boundary_marker, dirichlet_boundary_conditions,
                          c=c, p_initial=p_obs, degree=1, theta=0.5)
ana_solution = [s.copy() for s in ana_solution]

In [ ]:
pressures = {"p_init" : extract_cross_section(initial_solution, slice_points)/mmHg2Pa,
             "p_opt_dirichlet" : extract_cross_section(diri_opt_solution, slice_points)/mmHg2Pa,
             "p_opt_robin" : extract_cross_section(robin_opt_solution, slice_points)/mmHg2Pa,
             #"p_obs": extract_cross_section(p_obs, slice_points, times=times)/mmHg2Pa,
             "p_ana": extract_cross_section(ana_solution, slice_points)/mmHg2Pa
            }
#p_opt_robin = extract_cross_section(robin_opt_solution, slice_points)
#cdpdt = np.diff(p_opt_robin,n=1, axis=0, prepend=0)/dt*c

forces = {"f_opt_dirichlet": extract_cross_section(diri_opt_ctrls, slice_points),
          "f_opt_robin": extract_cross_section(robin_opt_ctrls, slice_points),
          "f_ana": extract_cross_section(f_ana, slice_points, times=np.array(times) - 0.5*dt),
          #"c*dp_opt_robin/dt":cdpdt
         }

style_dict["c*dp_opt_robin/dt"] = {"ls":":", "lw":5}

In [ ]:
%load_ext autoreload
%autoreload
from PlottingHelper import  plot_pressures_and_forces_cross_section
for i in range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, forces, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [2, 10,18]:
    plot_pressures_and_forces_timeslice(pressures, forces, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")